In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from irt.evaluation import eval_utils
from irt.data import data_loader
import matplotlib.pyplot as plt
import time
import girth
# from girth import rasch_conditional
from irt.data.rasch import generate_data
from irt.algorithms.spectral_estimator import spectral_estimate, construct_markov_chain, construct_markov_chain_accelerated
from irt.algorithms import conditional_mle
from irt.algorithms import rasch_mml
from irt.evaluation.eval_utils import log_likelihood_heldout, bayesian_auc, pairwise_disagreement_error, top_k_accuracy
import warnings
import torch as th
warnings.filterwarnings("ignore")

In [5]:
A = data_loader.riiid_small()
print(A.shape)
# print(len(ratings))

(6311, 1815)


In [4]:
dataset = "../experiment_results/may/may5/book_genome_m=9374_119_0.8"
# dataset = "../experiment_results/may/may5/bx_book_m=2185_119_0.8"

res_spectral = th.load(dataset + "_spectral.th")
res_mmle = th.load(dataset + "_MMLE.th")
res_jmle = th.load(dataset + "_JMLE.th")

In [5]:
beta_spec = res_spectral["est_spectral"]
rank_spec = np.argsort(beta_spec)[::-1]
beta_mmle = res_mmle["est_mmle"]
rank_mmle = np.argsort(beta_mmle)[::-1]
beta_jmle = res_jmle["est_jmle"]
rank_jmle = np.argsort(beta_jmle)[::-1]


In [6]:
sorted_ratings = sorted(ratings, key=lambda x: x[1], reverse=True)
true_rank = [item for (item, _, _) in sorted_ratings] # Sort from most popular items


In [7]:
K_array = [10, 25, 50, 75, 100, 200, 300]

cutoffs_top_k = [0, 50, 100, 250, 300, 400, 500]

for cutoff_rating in cutoffs_top_k:
    true_rank = [item for (item, _, num_ratings) in sorted_ratings if num_ratings > cutoff_rating]
    top_k_spectral = [top_k_accuracy(true_rank, rank_spec, K) for K in K_array]
    top_k_mmle = [top_k_accuracy(true_rank, rank_mmle, K) for K in K_array]
    top_k_jmle = [top_k_accuracy(true_rank, rank_jmle, K) for K in K_array]
    print()
    print(f"Min num ratings for top K = {cutoff_rating}")
    print("Spectral", top_k_spectral)
    print("MMLE", top_k_mmle)
    print("JMLE", top_k_jmle)




Min num ratings for top K = 0
Spectral [0.0, 0.0, 0.02, 0.013333333333333334, 0.04, 0.09, 0.13333333333333333]
MMLE [0.0, 0.0, 0.0, 0.0, 0.02, 0.05, 0.11666666666666667]
JMLE [0.0, 0.0, 0.0, 0.0, 0.01, 0.05, 0.10333333333333333]

Min num ratings for top K = 50
Spectral [0.0, 0.0, 0.02, 0.013333333333333334, 0.04, 0.09, 0.13333333333333333]
MMLE [0.0, 0.0, 0.0, 0.0, 0.02, 0.05, 0.11666666666666667]
JMLE [0.0, 0.0, 0.0, 0.0, 0.01, 0.05, 0.10333333333333333]

Min num ratings for top K = 100
Spectral [0.0, 0.0, 0.02, 0.013333333333333334, 0.04, 0.09, 0.13333333333333333]
MMLE [0.0, 0.0, 0.0, 0.0, 0.02, 0.05, 0.11666666666666667]
JMLE [0.0, 0.0, 0.0, 0.0, 0.01, 0.05, 0.10333333333333333]

Min num ratings for top K = 250
Spectral [0.0, 0.0, 0.02, 0.013333333333333334, 0.04, 0.045, 0.043333333333333335]
MMLE [0.0, 0.0, 0.0, 0.013333333333333334, 0.03, 0.015, 0.02666666666666667]
JMLE [0.0, 0.0, 0.0, 0.0, 0.02, 0.02, 0.03]

Min num ratings for top K = 300
Spectral [0.0, 0.0, 0.02, 0.04, 0.06,

In [8]:
res_spectral["top_k_spectral"]

[0.3, 0.16, 0.16, 0.18666666666666668, 0.18, 0.25, 0.32666666666666666]

In [29]:
cutoff_rating = 500

true_rank = [item for (item, _, num_ratings) in sorted_ratings if num_ratings > cutoff_rating]
top_k_spectral = [top_k_accuracy(true_rank, rank_spec, K) for K in K_array]
top_k_mmle = [top_k_accuracy(true_rank, rank_mmle, K) for K in K_array]
top_k_jmle = [top_k_accuracy(true_rank, rank_jmle, K) for K in K_array]


In [33]:
res_spectral["top_k_spectral"] = top_k_spectral
res_mmle["top_k_mmle"] = top_k_mmle
res_jmle["top_k_jmle"] = top_k_jmle

In [34]:
th.save(res_spectral, dataset + "_spectral.th")
th.save(res_mmle, dataset + "_MMLE.th")
th.save(res_jmle, dataset + "_JMLE.th")
